# Testing HDBSCAN on weighted adjacency matrices

This notebook is used to test the validity of the function hdbscan on graphs. To test it, we build the UMAP graph and we weight the edges with the low dimensional distances - after a full UMAP projection. We compare the results of HDBSCAN run on the low dimensional points with the results of the graph HDBSCAN version run on the partial distance (low-dim-distance-weighted graph).

## Conclusions:

The graph HDBSCAN code works well: so much better than what I had written myself. For this reason, many of the analysis I had run before (notebooks 07 and 07A) will need to be run again with this functi

In [1]:
!git branch

* master


In [1]:
execfile('functions/data_specifics.py')
execfile('functions/graph_functions.py')
print(data_set_list)

['pendigits', 'coil', 'mnist', 'usps', 'buildings', 'clusterable']


In [2]:
from IPython.display import display, Markdown, Latex
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, silhouette_score
from sklearn import cluster

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import umap
from collections import Counter
from scipy.stats import mode

from scipy.spatial.distance import euclidean

sns.set()

In [3]:
import hdbscan
import scipy.sparse as sp
import sklearn.cluster
from hdbscan._hdbscan_tree import (
    condense_tree,
    compute_stability,
    get_clusters,
    outlier_scores,
)
from hdbscan.plots import CondensedTree, SingleLinkageTree, MinimumSpanningTree

## Get clusters from a (disconnected) sparse distance matrix

* The 0-values indicate "far away" points

If the matrix has disconnected components (when viewed as a graph), HDBSCAN does not work. To get around this, we have tried clustering each component separately or just adding edges to glue the different parts together.

In [17]:
def hdbscan_on_graph(distance_graph):
    if scipy.sparse.csgraph.connected_components(distance_graph)[0] > 1:
        n_components, components = scipy.sparse.csgraph.connected_components(distance_graph)
        rj_labels = np.full(distance_graph.shape[0], -1, dtype=np.int64)
        for i in range(n_components):
            if np.sum(components == i) <= 90:
                subgraph_labels = np.zeros(np.sum(components == i), dtype=np.int64)
            else:
                subgraph = distance_graph[components == i, :][:, components == i]
                subgraph_labels = hdbscan.HDBSCAN(metric="precomputed", min_samples=2, min_cluster_size=30, allow_single_cluster=subgraph.shape[0] < 300).fit_predict(subgraph)
            subgraph_labels[subgraph_labels >= 0] += (rj_labels.max() + 1)
            rj_labels[components == i] = subgraph_labels
            # print(i, np.sum(components == i), Counter(subgraph_labels))
    else:
        rj_labels = hdbscan.HDBSCAN(metric="precomputed", min_samples=2, min_cluster_size=30).fit_predict(distance_graph)
    return(rj_labels)

# Clustering algorithm
These are the steps we use to feed a sparse distance matrix to the single linkage...
We are not using HDBSCAN at the moment, but this is where we would like to go.

## Get data, UMAP graph and UMAP low dimensional vectors

In [5]:
dataset_id=0
raw_data, targets, dataset_name = get_dataset(dataset_id=dataset_id)

k = get_dataset_params(dataset_id)['n_neighbors']

A_umap, sigmas, rhos, dists = umap.umap_.fuzzy_simplicial_set(X=raw_data, 
                                             n_neighbors=k, 
                                             random_state=0, 
                                             metric='euclidean', 
                                             return_dists=True,
                                             set_op_mix_ratio=1)

A_knn = knn_adjacency(raw_data, k=k)

umap_rep = get_umap_vectors(dataset_id=dataset_id, raw_data=raw_data)

## Build the partial distance matrix based of UMAP graph

In [6]:
# Symmetric D
D = A_umap.copy()
rows = [x for v in [[i]*v for i, v in enumerate(A_umap.indptr[1:] - A_umap.indptr[:-1])] for x in v]
D.data = np.array([euclidean( umap_rep[rows[i]], 
                                         umap_rep[A_umap.indices[i]])
            for i in range(len(rows))
           ])

## Compare robust single-linkage (portions of HDBSCAN) on partial distance vs. on low dim projection

In [8]:
labels = hdbscan_on_graph(D)

ari = adjusted_rand_score(targets, labels)
ami = adjusted_mutual_info_score(targets, labels)
print(f'ARI = {ari} and AMI = {ami}')

ARI = 0.9065482347624224 and AMI = 0.9238583306873789


In [9]:
hd_umap_labels = h_dbscan(umap_rep, which_algo='hdbscan', dataset_id=dataset_id)

ari = adjusted_rand_score(targets, hd_umap_labels)
ami = adjusted_mutual_info_score(targets, hd_umap_labels)
print(f'ARI = {ari} and AMI = {ami}')

ARI = 0.9185149200427103 and AMI = 0.9320899303214291


The difference between the HDBSCAN results on the low-dimensional dataset and HDBSCAN results on the partial distance matrix limited to the UMAP graph entries is huge. Is this normal or is it some issues with my HDBSCAN code?

# Add mutual reachability

In [10]:
D_sparse_lil = D.tolil()
D_mr = hdbscan.hdbscan_.sparse_mutual_reachability(D_sparse_lil)

labels = hdbscan_on_graph(D_mr)

ari = adjusted_rand_score(targets, labels)
ami = adjusted_mutual_info_score(targets, labels)
print(f'ARI = {ari} and AMI = {ami}')

ARI = 0.9040986701693291 and AMI = 0.9209138850765751


## Non-symmetrical matrices

In [11]:
D_sparse_lil = dists.tolil()
D_mr = hdbscan.hdbscan_.sparse_mutual_reachability(D_sparse_lil)

labels = hdbscan_on_graph(D_mr)

ari = adjusted_rand_score(targets, labels)
ami = adjusted_mutual_info_score(targets, labels)
print(f'ARI = {ari} and AMI = {ami}')

ARI = 0.39813849659122247 and AMI = 0.7081322939635483


In [12]:
# Non-Symmetric D
rows, cols = A_knn.nonzero()
data = np.array([euclidean( umap_rep[rows[i]], umap_rep[cols[i]] ) for i in range(len(rows))])
D_knn = sp.coo_matrix((data, (rows, cols)), shape=A_knn.shape)

In [13]:
D_sparse_lil = D_knn.tolil()
D_mr = hdbscan.hdbscan_.sparse_mutual_reachability(D_sparse_lil)

labels = hdbscan_on_graph(D_mr)

ari = adjusted_rand_score(targets, labels)
ami = adjusted_mutual_info_score(targets, labels)
print(f'ARI = {ari} and AMI = {ami}')

ARI = 0.9031072081545124 and AMI = 0.9199781241822763


# Run on all datasets

In [14]:
def get_single_linkage_clustering(dataset_id, distance_function, k=None):
    raw_data, targets, dataset_name = get_dataset(dataset_id=dataset_id)
    display(Markdown(f'### {dataset_name}'))
    
    if(distance_function.__name__ == "get_avg_neighbor_distance"):
        D = distance_function(raw_data)
    else:
        if(k is None):
            k = get_dataset_params(dataset_id)['n_neighbors']

        A_umap, sigmas, rhos, dists = umap.umap_.fuzzy_simplicial_set(X=raw_data, 
                                                         n_neighbors=k, 
                                                         random_state=0, 
                                                         metric='euclidean', 
                                                         return_dists=True,
                                                         set_op_mix_ratio=0.5)
        umap_rep = get_umap_vectors(dataset_id=dataset_id, raw_data=raw_data)
        D = distance_function(A_umap, umap_rep)

    labels = hdbscan_on_graph(D)

    ari = adjusted_rand_score(targets, labels)
    ami = adjusted_mutual_info_score(targets, labels)
    print(f'PARTIAL DISTANCE:\nARI = {ari} and AMI = {ami}\n')
    
    hd_umap_labels = h_dbscan(umap_rep, which_algo='hdbscan', dataset_id=dataset_id)

    ari = adjusted_rand_score(targets, hd_umap_labels)
    ami = adjusted_mutual_info_score(targets, hd_umap_labels)
    print(f'UMAP+HDBSCAN:\nARI = {ari} and AMI = {ami}\n\n')
    
    return(labels, ari, ami, targets)

In [18]:
def low_dim_distance(A_umap, umap_rep):
    D = A_umap.copy()
    rows = [x for v in [[i]*v for i, v in enumerate(A_umap.indptr[1:] - A_umap.indptr[:-1])] for x in v]
    D.data = np.array([euclidean( umap_rep[rows[i]], 
                                             umap_rep[A_umap.indices[i]])
                for i in range(len(rows))
               ])
    return(D)

In [19]:
for i in range(6):
    res = get_single_linkage_clustering(dataset_id=i, distance_function = low_dim_distance)

### pendigits

PARTIAL DISTANCE:
ARI = 0.9065482347624224 and AMI = 0.9238583306873789

UMAP+HDBSCAN:
ARI = 0.9185149200427103 and AMI = 0.9320899303214291




### coil

/work/home/vmpouli/.conda/envs/HighDimensionalClustering/lib/python3.7/site-packages/sklearn/manifold/_spectral_embedding.py:261: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  "Graph is not fully connected, spectral embedding may not work as expected."


PARTIAL DISTANCE:
ARI = 0.8087548896884135 and AMI = 0.9506610006990772

UMAP+HDBSCAN:
ARI = 0.7794373675775127 and AMI = 0.9370434150630383




...

PARTIAL DISTANCE:
ARI = 0.8823920433016351 and AMI = 0.8997656607126547

UMAP+HDBSCAN:
ARI = 0.8824676944734986 and AMI = 0.9004545663772475




### buildings

PARTIAL DISTANCE:
ARI = 0.22386999014479378 and AMI = 0.5875305104687952

UMAP+HDBSCAN:
ARI = 0.25147767218321093 and AMI = 0.6310982061281075




### clusterable

PARTIAL DISTANCE:
ARI = 0.25073427516095526 and AMI = 0.5597120405889396

UMAP+HDBSCAN:
ARI = 0.14648646266389906 and AMI = 0.43229025916559943


